# Partial Replication of S&P 500 Equal Weight Index

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

## Data Extraction

In [2]:
def process_data_chunked(file_path, process):
    dataset_list = []

    for df in pd.read_csv(file_path, index_col=0, header=[0, 1], chunksize=1000, na_values=['', 'NaN', 'NULL']):
        df_processed = process(df)
        dataset_list.append(df_processed)
    return pd.concat(dataset_list)

In [3]:
def keep_close(df):
    return df.xs('close', level=1, axis=1)

In [4]:
def tracking_error(index, predictions):
    return np.std(pd.Series(index).pct_change()*100 - pd.Series(predictions).pct_change()*100)

In [5]:
df = process_data_chunked('/kaggle/input/snp-500-intraday-data/dataset.csv', keep_close)
df.head(3)

,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,...,XL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZION,ZTS
timestamp,,,,,,,,,,,,,,,,,,,,,
2017-09-11 09:30:00,NaN,44.01,NaN,160.57,84.530,83.24,NaN,134.00,156.590,80.810,...,41.050,64.0400,79.41,58.7600,31.7500,63.07,76.82,116.25,42.04,NaN
2017-09-11 09:31:00,65.46,44.25,93.565,160.51,84.555,82.89,52.43,134.45,156.995,81.260,...,41.950,64.1833,79.41,58.6527,31.6900,63.16,76.84,116.25,42.24,65.11
2017-09-11 09:32:00,65.66,44.27,94.060,160.32,84.940,NaN,52.50,134.53,156.850,81.185,...,42.235,64.0200,79.12,NaN,31.6997,62.80,76.90,116.74,42.48,65.12


## Data Cleaning

In [6]:
df.interpolate(method='linear', inplace=True)
# To fill the first row NaN with the value from the next minute
df.bfill(inplace=True)
df.head(3)

,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,...,XL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZION,ZTS
timestamp,,,,,,,,,,,,,,,,,,,,,
2017-09-11 09:30:00,65.46,44.01,93.565,160.57,84.530,83.240,52.43,134.00,156.590,80.810,...,41.050,64.0400,79.41,58.76000,31.7500,63.07,76.82,116.25,42.04,65.11
2017-09-11 09:31:00,65.46,44.25,93.565,160.51,84.555,82.890,52.43,134.45,156.995,81.260,...,41.950,64.1833,79.41,58.65270,31.6900,63.16,76.84,116.25,42.24,65.11
2017-09-11 09:32:00,65.66,44.27,94.060,160.32,84.940,82.975,52.50,134.53,156.850,81.185,...,42.235,64.0200,79.12,58.76135,31.6997,62.80,76.90,116.74,42.48,65.12


In [7]:
# Create target variable - SPXEW value
df['SP500_EW'] = df.sum(axis=1) / len(df.columns)
# df['SP500_EW_Forward_Diff'] = df['SP500_EW'].shift(-1) - df['SP500_EW']
df.dropna(inplace=True)
df = df.shift(-1)
df.head(3)

,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,...,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZION,ZTS,SP500_EW
timestamp,,,,,,,,,,,,,,,,,,,,,
2017-09-11 09:30:00,65.46,44.25,93.5650,160.51,84.555,82.890,52.4300,134.45,156.995,81.260,...,64.1833,79.41,58.65270,31.6900,63.16,76.84,116.25,42.24,65.110,101.098165
2017-09-11 09:31:00,65.66,44.27,94.0600,160.32,84.940,82.975,52.5000,134.53,156.850,81.185,...,64.0200,79.12,58.76135,31.6997,62.80,76.90,116.74,42.48,65.120,101.141250
2017-09-11 09:32:00,65.69,44.34,94.0775,160.15,84.990,83.060,52.4986,134.61,156.780,81.440,...,64.0900,79.19,58.87000,31.7400,62.88,76.89,116.27,42.20,65.045,101.180133


In [8]:
df['SP500_EW_Shifted'] = df['SP500_EW'].shift(-1)

In [9]:
df.dropna(inplace=True)

In [10]:
df.head(2)

,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,...,XOM,XRAY,XRX,XYL,YUM,ZBH,ZION,ZTS,SP500_EW,SP500_EW_Shifted
timestamp,,,,,,,,,,,,,,,,,,,,,
2017-09-11 09:30:00,65.46,44.25,93.565,160.51,84.555,82.890,52.43,134.45,156.995,81.260,...,79.41,58.65270,31.6900,63.16,76.84,116.25,42.24,65.11,101.098165,101.141250
2017-09-11 09:31:00,65.66,44.27,94.060,160.32,84.940,82.975,52.50,134.53,156.850,81.185,...,79.12,58.76135,31.6997,62.80,76.90,116.74,42.48,65.12,101.141250,101.180133


## Models

### Linear Regression

In [11]:
X = df.drop(columns=['SP500_EW', 'SP500_EW_Shifted'])
y = df['SP500_EW']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

lasso = Lasso(alpha=1)
lasso.fit(X_train, y_train)

y_pred_train = lasso.predict(X_train)
y_pred_test = lasso.predict(X_test)

mse_train = mean_squared_error(y_train, y_pred_train)
r2_train = r2_score(y_train, y_pred_train)

mse_test = mean_squared_error(y_test, y_pred_test)
r2_test = r2_score(y_test, y_pred_test)

print("Training Mean Squared Error:", mse_train)
print("Training R² Score:", r2_train)
print("Testing Mean Squared Error:", mse_test)
print("Testing R² Score:", r2_test)

Training Mean Squared Error: 0.16370849442671428
Training R² Score: 0.9880025054352332
Testing Mean Squared Error: 0.16665920977283408
Testing R² Score: 0.9879077777577618


In [12]:
coef = lasso.coef_
features = X.columns

# Create a DataFrame for the coefficients
coef_df = pd.DataFrame({'Stock': features, 'Coefficient': coef})

# Filter for non-zero coefficients
non_zero_coef_df = coef_df[coef_df['Coefficient'] != 0]

chosen_stocks = non_zero_coef_df['Stock'].values
print(len(chosen_stocks))

predicted = lasso.predict(X_train)

14


In [13]:
predicted

array([107.29909869, 102.13716759, 104.68073687, ..., 113.03447494,
       101.74746287, 104.03327464])

In [14]:
y_train.values

array([105.9494207 , 102.5161402 , 105.2909906 , ..., 113.34046305,
       101.6258689 , 103.75361382])

In [15]:
np.std(pd.Series(predicted).pct_change()*100 - pd.Series(y_train.values).pct_change()*100)

0.5451988281150716